In [7]:
import os
import sys, os
import dill
import sqlalchemy
import ipywidgets as widgets
from aepsych.plotting import plot_strat
from aepsych.server import AEPsychServer
from aepsych.server.sockets import PySocket
from IPython.display import display
from ipywidgets import HTML, AppLayout, Box, Button, Label, Layout, Text
from IPython.core.display import HTML as html

# print(sys.path)

connect_out = widgets.Output(layout=Layout(padding="2px", height="150px"))


with connect_out:
    server = AEPsychServer()
    connect_out.clear_output()
strat = None
db_file_name = None
db_config_str = None
database_path = None
# dim = 0


#---------- Style ----------

display(html("""<style>
.jp-CodeCell {
    width:100% !important;
    background: whitesmoke;
}
.jp-Notebook .jp-Cell {
    padding: 0 !important;
}

.jupyter-widgets.widget-tab {
    margin: 40px 30px;
}

.p-Collapse-header,
.jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab.p-mod-current:before,
.jupyter-button ,
.widget-button,
.jupyter-widgets.widget-tab > .widget-tab-contents {
    border-radius: 4px;
    border: 1px solid #dcdcdc;
}


.jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab{
border-top-right-radius:4px;
border-top-left-radius: 4px;
border: 1px solid #dcdcdc;
}
.p-Collapse-contents{
border-bottom-right-radius:4px;
border-bottom-left-radius: 4px;
border: 1px solid #dcdcdc;
border-top:none;
}

.p-Collapse-open > .p-Collapse-header {
border-bottom-right-radius:0px;
border-bottom-left-radius: 0px;
}
body.jp-Notebook {
    margin: 0 !important;
    padding: 0 !important;
    height: 100vh !important;
    background: whitesmoke;
}
.widget-inline-hbox {
    margin: 0px;
}
.widget-output {
   # padding: 10px !important;
   # background: white;
   # border-radius: 4px !important;
   # box-shadow: inset 0 1px 1px rgb(0 0 0 / 5%);
   # border: 1px solid #e3e3e3;
   # border-radius: 4px;
   # -webkit-box-shadow: inset 0 1px 1px rgb(0 0 0 / 5%);
   # box-shadow: inset 0 1px 1px rgb(0 0 0 / 5%);
   # margin: 10px;
   # max-width: 600px; 
    height: auto !important;
    width: auto !important;
   # overflow-y: auto;
}

.jp-OutputArea {
    overflow-y: inherit;
}

.widget-text input[type="text"],
.widget-text input[type="number"],
.widget-dropdown > select {
   border-radius: 4px;
   box-shadow: inset 0 1px 1px rgb(0 0 0 / 5%);
}

.widget-vbox {
    justify-content: center;
    width: 100%;
    width: -webkit-fill-available !important;
    display: flex;
    border-radius: 4px;
    background: white;
    height: fit-content;
    padding: 20px;
   # box-shadow: 0px 1px 3px rgb(0 0 0 / 12%), 0px 1px 2px rgb(0 0 0 / 24%);
}


[data-jp-theme-light='true'] .jp-RenderedImage img.jp-needs-light-background {
    width: 100%;
}


.jupyter-widgets.widget-tab {
    min-width: 30% !important;
}
html {
    background: #F2F2F2;
    height: 100%;
    margin: 0px;
    padding: 0px;

}

.widget-dropdown,
.widget-text{
    width: auto;

}
.widget-label {
    display: flex;
    align-items: center;
}
.jp-RenderedText[data-mime-type='application/vnd.jupyter.stderr'] {
    background: white;
    padding-top: var(--jp-code-padding);
}
.jupyter-widgets.widget-tab > .widget-tab-contents {
    border-radius: 4px;
    border-top-left-radius: 0px;
    border-top-right-radius: 0px;
}
.widget-textarea textarea {
    height: auto;
    width: auto;
    min-height: 100%;
    resize: none;
    background-color:black; 
    color: green;
}
.widget-box {
    overflow: hidden !important;
}
.jp-OutputArea-output {
    overflow: hidden
    }

</style>"""))


header = HTML(
    """
<div class="nav" style=" 
margin: 0px;
background-color: #F2F2F2;
font-weight: bold; 
box-shadow: 0px 1px 3px rgb(0 0 0 / 12%), 0px 1px 2px rgb(0 0 0 / 24%);
">
    <h1 style="
    text-align: center;
    font-family: helvetica;
    font-weight: 400;
    font-size: 20px;
    display: flex;
    align-items: center; 
    padding: 15px 20px;
    margin: 0px;
    hieght: 100px; 
    jusify-content: flex-start;
    color:rgb(80,103,132)">
    <img src="./logo.png" alt="logo" width="50" height="50" style="margin-right: 10px;" >
    AEPsych Visualizer
    </h1>
</div>
"""
)

logs_style = Layout(width='600px', height='400px')
input_style = {"description_width": "initial"}
input_layout= Layout(margin="5px 0")
btn_style = Layout(margin="10px",width="95%")

btn_box_layout = Layout(display="flex", justify_content="flex-end")
file_output = widgets.Output(
    layout={
            "overflow": "scroll", 
            "height": "50vh",
            "width": "100%",
            "padding": "5px"
           }
)

# ---------- Inputs -----------

def on_value_change(change):
    file_output.clear_output()
    with file_output:
        change["new"]
#         print(change['new'].keys())
#         print(change["new"])

        
def on_value_change_input(change):
    inputs_output.clear_output()
    with inputs_output:
        change["new"]   


uploader = widgets.FileUpload(
    description="Select File",accept=".db", multiple=False, layout=btn_style)

yes_label = Text(
    value="Yes Trial", placeholder="yes_label",
    description="yes_label: ", disabled=False, style=input_style,
    layout=input_layout
)

no_label = Text(
    value="No Trial", placeholder="no_label", 
    description="no_label: ", disabled=False, style=input_style,
    layout=input_layout
)

target_level = widgets.BoundedFloatText(
    value=0.75, min=0, max=1, 
    step=0.1, description="target_level:", disabled=False, 
    style=input_style, layout=input_layout
)

cred_level = widgets.BoundedFloatText(
    value=0.95, min=0, max=1, step=0.1, 
    description="cred_level:", disabled=False, 
    style=input_style, layout=input_layout
)

xlabel = Text(
    value="Angle (degrees)", placeholder="x axis label",
    description="xlabel: ", disabled=False, style=input_style,
    layout=input_layout
)

ylabel = Text(
    value="Detection Probability", placeholder="y axis label", 
    description="ylabel: ", disabled=False, style=input_style,
    layout=input_layout
)

# Observes input changes
uploader.observe(on_value_change, names="value")

inputs_output = widgets.Output(
    layout={"border": "1px solid black", "padding": "20px", "margin": "20px", "min-height ": "100px"}
)
yes_label.observe(on_value_change_input, names="value")
no_label.observe(on_value_change_input, names="value")
target_level.observe(on_value_change_input, names="value")
cred_level.observe(on_value_change_input, names="value")
xlabel.observe(on_value_change_input, names="value")
ylabel.observe(on_value_change_input, names="value")


# -------- ports ----------
server_ip = widgets.Text(
    placeholder="0.0.0.0", # must use regex to check pattern and type int
    description='Server IP',
    disabled=False,
    layout=Layout(display='flex', flex_flow="column", margin="10px"))
port = widgets.Text(
    placeholder="5555", # must add regex, type check, and error handling
    description='Port',
    disabled=False,
    layout=Layout(display='flex', flex_flow="column", margin="10px", justify_content="center"))

connected_ip = widgets.Output()

def on_change_server_input(change):
    with inputs_output:
        change["new"]

server_ip.observe(on_change_server_input, names="value")
port.observe(on_change_server_input, names="value") 

button_submit_pysocket = widgets.Button(description="submit", disabled=False, 
    button_style="info", tooltip="submit")



def submit_pysocket_clicked(b):
    global server 
    connected_ip.clear_output()
    with connected_ip:
#         try: 
        server = AEPsychServer(database_path=database_path)
        load_logs()
        print( f"Connected by Client at {server_ip.value}")
#         except Exception as e:
# #             print("")
#             print(e)


button_submit_pysocket.on_click(submit_pysocket_clicked)

connection_inputs = widgets.Box([server_ip, port,button_submit_pysocket], 
                               layout=Layout(justify_content="flex-end",margin="4px"))


# -------- Config --------
default_config_file = None


config_output = widgets.Output(layout=Layout(margin="10px"))

def update_config_value():
    global db_config_str
    config_output.clear_output()
    with config_output:
        display(html(f"""
        <div style='height: 400px;'>
            <pre
            style='font-family: monospace;background-color: black;
            color: green;padding: 8px;height: 100%;overflow: auto; '
            >{db_config_str}</pre>

        </div>
        """))


def show_config():
    '''
    Displays database config. (View only)
    '''
    global server
    config = config_textarea.value
    config_output.clear_output()
#     try: 
#         with config_output:
            
# #             server.configure(config_str=config) # New confgi
# #             print("Submit, need to add functionality to update config in db?")
#     except Exception as e:
# #             print("Unable to configure server. Please reset and reconfigure.")
#             print(e)

                
# def clicked_submit_config(b):
#     show_config()
    
    

# def clicked_reset_config(b):
#     global default_config_file 
#     config_output.clear_output()
#     config_textarea.value = default_config_file
    
    
# button_submit_config.on_click(clicked_submit_config)
# button_reset_config.on_click(clicked_reset_config)

# -------- Plot view ----------

# --------- Accordion -----------
button_submit = widgets.Button(
    description="Submit", disabled=False, 
    button_style="info", tooltip="submit", 
    layout=btn_style
)
accordion = widgets.Accordion(
    children=[
        widgets.VBox([no_label, yes_label], layout=Layout(padding="5px")),
        widgets.VBox(
            [
                target_level,
                cred_level,
            ],
            layout=Layout(padding="5px")
        ),
        widgets.VBox([xlabel, ylabel],layout=Layout(padding="5px")),
    ],
    layout=Layout(min_width="280px")
)
accordion.set_title(0, "Outcome Labels")
accordion.set_title(1, "Parameters")
accordion.set_title(2, "Axis Labels")




def submit():
    file_output.clear_output()
    with file_output:
        display_plot()
        
        
def submit_clicked(b):
    submit()

    
button_submit.on_click(submit_clicked)

plot_interact_box = widgets.VBox([accordion, button_submit])

# -------- Logging --------
import logging
import aepsych.utils_logging as utils_logging

logger_output = widgets.Output()

log_path = os.getcwd() + "/logs/bayes_opt_server.log"


# def load_logs():
#     logger_output.clear_output()
#     with logger_output:
#         with open(log_path) as f:
#             contents = f.readlines()
#             for log in contents:
#                 print(log)

def load_logs():
    logger_output.clear_output()
    log_str = ""
    logger_output.clear_output()
    with logger_output:
        with open(log_path) as f:
            contents = f.readlines()
            for log in contents:
                # print(log)
                log_str += f"<p style='margin-top: 1px;margin-bottem: 1px'>{log}</p>"
        display(html(f"""
        <div style='height: 400px;overflow: auto;'>
            <pre
            style='font-family: monospace; background-color: black; color: white; 
            padding: 4px; height: 100%;'
            >{log_str}</pre>
        </div>
        """))
                

# --------- Interactive ------------

param_output = widgets.Output()

# def ask_for_params():
#     global strat
#     global server
#     print(strat.gen()[0])
#     trial_params = server.ask()
#     with param_output:
#         print("ask--->",trial_params)

        
        
# --> Tab 1 
param_one = widgets.FloatText(
    value=1.23,
    description='Parameter1: ',
    disabled=False,
    layout=Layout(margin="5px 10px")
)
param_two = widgets.FloatText(
    value=3.45,
    description='Parameter2: ',
    disabled=False,
    layout=Layout(margin="5px 10px")
)
outcome = widgets.IntText(value=0, description='Outcome: ', 
                          disabled=False,layout=Layout(margin="5px 10px"))

param_box_tab = widgets.VBox([param_output, param_one, param_two, outcome], 
                         layout=Layout(max_width="250px"))

# --> Tab 2



# --> Tab 3


tab_labels = ['New Parameters', 'Update Model', 'Send to Client']
tab_widgets = [param_box_tab, Label('Update Model'), Label('Send to Client')]

children = [widget for widget in tab_widgets]
interactive_tabs = widgets.Tab(layout=Layout(width='100%', margin="0"))
interactive_tabs.children = children
for i in range(len(children)):
    interactive_tabs.set_title(i, tab_labels[i])

    
# ------- View 2 --------
# --> plot view 
plot_output = widgets.Output(layout=Layout(
    padding="2px", 
    height="100%", 
    align_items='center',
    justify_conten="center",
    width="100%"))

# --> table view 

import pandas as pd
import numpy as np

table_data_output = widgets.Output(layout=Layout(width='600px',height='400px',))



# def display_data():
#     global db_config_str
#     with table_data_output:
#         uuid = server.db.get_master_records()[0].experiment_id
#         strat = server.get_strat_from_replay()
#         replay_data = server.db.get_replay_for(uuid)
#         print('{:<4}{:>20}{:>20}{:>25}'.format("unique_","message_type","master_talbe_id","message_contents"))

#         for record in replay_data:
#             if record.message_type == 'setup':   
#                 db_config_str = record.message_contents['message']['config_str']
#                 update_config_value()
#                 print('{:<4}{:>20}{:>20}{:>25}'.format(record.unique_id, 
#                       record.message_type, 
#                       record.master_table_id, 
#                       'view config under Exp details',)) 
#             else: 
#                 print('{:<4}{:>20}{:>20}{:>25}'.format(record.unique_id, 
#                       record.message_type, 
#                       record.master_table_id,
#                       str(record.message_contents['message'])))
            
    
        
#         data = server.get_dataframe_from_replay()
#         display("data", data)

def display_data():
    global db_config_str
    with table_data_output:
        uuid = server.db.get_master_records()[0].experiment_id
        replay_data = server.db.get_replay_for(uuid)
        setup_table_row = ""
        table_rows = ""

        for record in replay_data:
            if record.message_type == 'setup':
                db_config_str = record.message_contents['message']['config_str']
                setup_table_row += f"""
                  <tr>
                    <th>{record.unique_id}</th>
                    <th>{record.message_type}</th>
                    <th>{record.master_table_id}</th>
                    <th>view config under Exp details tab</th>
                </tr>
                """
            else: 
                table_rows += f"""
                  <tr>
                    <th>{record.unique_id}</th>
                    <th>{record.message_type}</th>
                    <th>{record.master_table_id}</th>
                    <th>{str(record.message_contents['message'])}</th>
                </tr>
                """
        display(html(f"""
          <div style='height: 400px; overflow: auto;'>
            <table style='width: 100%;'>
                <tr>
                    <th>unique_</th>
                    <th>message_type</th>
                    <th>master_talbe_id</th>
                    <th>message_contents</th>
                </tr>
                {setup_table_row}
                {table_rows}
            </table>
          </div>
        """))

        
# ---> use for new session 
def display_plot():
    global strat
    plot_output.clear_output()
    with plot_output:
#     #------- Error Handling -----
#         error = False
#         with plot_output:
#             plot_output.clear_output()
#             if len(ylabel.value) > 35: 
#                 error = True
#     #             raise Exception("ylabel must be less than 35 characters ")
#                 print("ylabel must be less than 35 characters ")
#             elif len(xlabel.value) > 35: 
#                 error = True
#                 print("xlabel must be less than 35 characters ")
#             elif len(yes_label.value) > 35: 
#                 error = True
#                 print("yes_label must be less than 35 characters ")
#             elif len(no_label.value) > 35: 
#                 error = True
#                 print("no_label must be less than 35 characters ")

#         if error == False: 
#             inputs_output.clear_output()
        # ----  input value  New session 
        if strat != None and db_file_name == None:
            plot_strat(
                strat,
                xlabel=xlabel.value,
                ylabel=ylabel.value,
                no_label=no_label.value,
                yes_label=yes_label.value,
                cred_level=cred_level.value,
                target_level=target_level.value,
            )
            return True
        # ---- value from db      
        if strat != None and db_file_name != None:
            print(
#                 "server.strat",
#                 dir(server),
#                server.strat.lb, 
#                server.strat.ub
#                   strat,
#                   strat.dim,
#                   server.strat._strat_idx,
#                   server.parnames[0]
                 )
            plot_strat(
                strat,
                xlabel=xlabel.value,
                ylabel=ylabel.value,
                no_label=no_label.value,
                yes_label=yes_label.value,
                cred_level=cred_level.value,
                target_level=target_level.value,
            )
            return True
#             if server.strat.dim <= 2:
#                 if server.strat._strat_idx > 0:
#                     xlabel = server.parnames[0]
#                     ylabel = server.parnames[1] if server.strat.dim == 2 else None
#                     yes_label = yes_label.value
#                     no_label = no_label.value
# #                     acqf = server.strat._strat.generator.acqf
#                     thresh = target_level.value
#                     plot_strat(
#                         server.strat, xlabel=xlabel, ylabel=ylabel, target_level=thresh,
#                         yes_label=yes_label, no_label=no_label
#                     )
#                 else:
#                     print(
#                         "\n\n\n\n\n Initializing model. Collect more data to plot posterior."
#                     ) 
#                     return True 
#             else:
#                 print("Plotting currently only works for <=2D")
#                 return True 
        else:
            print("Server has no experiment records!")
            return False 

out = widgets.Output(layout=Layout(
    padding="2px", 
    height="100%", 
    align_items='center',
    justify_conten="center",
    margin="0",
    width="100%"))

exp_details = widgets.Button(
    description='Exp Details',
    disabled=False,
    button_style='',
    tooltip='Click me')
plot_view = widgets.Button(
    description="Plot View",
    disable=False,
    button_style="",
    tooltip="Click me")
table_view = widgets.Button(
    description="Table View",
    disable=False,
    button_style="",
    tooltip="Click me")
logs = widgets.Button(
    description="Logs",
    disable=False,
    button_style="",
    tooltip="Click me")
interactive = widgets.Button(
    description="Interactive",
    disable=False,
    button_style="",
    tooltip="Click me")

navigation = widgets.Box([exp_details,plot_view,table_view,logs,interactive],
                         
                         
layout=Layout(  
        display='flex',
        flex_flow="column",
        width='200px',
        margin="0 2.5%",
        min_width="152px"
        ))
output_header_style = Layout(justify_content="center")
nav_display_style = Layout(display="flex",
        flex_flow="column")


container_one = widgets.Box([
    Label("Exp Details", layout=output_header_style),
    config_output],
    layout=nav_display_style )
container_two = widgets.Box([
    Label("Plot View", layout=output_header_style),
    widgets.HBox([plot_output,plot_interact_box])

    ],
    layout=Layout(max_width="auto",display="flex",flex_flow="column"))
container_three = widgets.Box([
    Label("Table View"),table_data_output],
    layout=nav_display_style)
container_four = widgets.Box([
    Label("Logs", layout=output_header_style),
    logger_output],
    layout=nav_display_style)

container_five = widgets.Box([interactive_tabs], layout=Layout(margin="0",padding="0"))

nav_container = widgets.Box(
    [navigation,out],
    layout=widgets.Layout(
    display="flex",
    flex_flow='row'
    ))
main_container = widgets.Box(
    [connection_inputs,nav_container],
    layout=widgets.Layout(
    display="flex",
    flex_flow="column",
    marging="auto"
    )
)

# display(main_container)


def nav_btn_click(b):
    show_display = b.description
    out.clear_output()
    if show_display == 'Exp Details':
        with out:
            display(container_one)
    elif show_display == "Plot View":
        with out:
            display_plot()
            display(container_two)
    elif show_display == "Table View":
        with out: 
            display(container_three)
    elif show_display == "Logs":
        with out:
            load_logs()
            display(container_four)
    elif show_display == "Interactive":
#         ask_for_params()
        with out: 
            display(container_five)

            
exp_details.on_click(nav_btn_click)    
plot_view.on_click(nav_btn_click)
table_view.on_click(nav_btn_click)
logs.on_click(nav_btn_click)
interactive.on_click(nav_btn_click)


#------ View 1 ------            
uploader = widgets.FileUpload(
    description="Connect to Database",accept=".db", multiple=False, layout=Layout(margin="15px 60px"))

def start_server():
    config = show_config()
    global strat
    global server
    with error_output:
        try:
            server = AEPsychServer(database_path=database_path)
            strat = server.get_strat_from_replay()
            print(lent(strat))
            display_data()
            if display_plot() == True:
                home_view_container.layout.display = 'none' 
                main_container.layout.display = 'inline'
            else:
                error_output.clear_output()
                if strat != None:
                    try:
                        plot_strat(
                            strat,
                            xlabel=xlabel.value,
                            ylabel=ylabel.value,
                            no_label=no_label.value,
                            yes_label=yes_label.value,
                            cred_level=cred_level.value,
                            target_level=target_level.value,
                        )
                    except RuntimeError as err: 
                        error_output.clear_output()
                        with error_output:
                            print("Unable to connect to server. \n")
                            print(err)
                    except AssertionError as err: 
                        print(err)

                display(main_container)
        except sqlalchemy.exc.DatabaseError as err:
                    error_output.clear_output()
                    with error_output:
                        print("Unable to connect to server. \n")
                        print(err)
        except Exception as e:
            print("unable to connect to server...")
            print(e)
    #             exc_type, exc_obj, exc_tb = sys.exc_info()
    #             fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
    #             print(exc_type, fname, exc_tb.tb_lineno)    
    
    
    
def resume_server():
    global strat
    global server
    with error_output:
        try:
            !true > logs/bayes_opt_server.log
            server = AEPsychServer(database_path=database_path)
            strat = server.get_strat_from_replay()
            display_data()
#           clears the server.log file data when you start
#             if display_plot() == True:
            update_config_value()
            with out:
                display(container_one)
            home_view_container.layout.display = 'none' 
            main_container.layout.display = 'inline'
#             else:
#                 error_output.clear_output()
#                 if strat != None:
#                     try:
#                         plot_strat(
#                             strat,
#                             xlabel=xlabel.value,
#                             ylabel=ylabel.value,
#                             no_label=no_label.value,
#                             yes_label=yes_label.value,
#                             cred_level=cred_level.value,
#                             target_level=target_level.value,
#                         )
#                     except RuntimeError as err: 
#                         error_output.clear_output()
#                         with error_output:
#                             print("Unable to connect to server. \n")
#                             print(err)
#                     except AssertionError as err: 
#                         print(err)

#                 display(main_container)
#         except sqlalchemy.exc.DatabaseError as err:
#                     error_output.clear_output()
#                     with error_output:
#                         print("Unable to connect to server. \n")
#                         print(err)
        except Exception as e:
            print("unable to connect to server...")
            print(e)
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            print(exc_type, fname, exc_tb.tb_lineno)
            

def upload():
    global server
    global strat
    global database_path
    global db_file_name
    error_output.clear_output() 
    with error_output:     
        error_output.clear_output()
        if uploader.value == {}:
            print("No file uploaded")
        else:
            for file_name, file in uploader.value.items():
                db_file_name = file_name 
                current_path = os.getcwd()
                external_target_path = current_path + "/databases"
                completeName = None 
                
                if os.path.isdir(external_target_path):
                    completeName = os.path.join(external_target_path, file_name)
                else:
                    print("File upload unsuccessful")

                with open(completeName, "wb") as f:
                    f.write(file["content"])
                    if completeName != None:
                        database_path = completeName
                    resume_server()
                    f.close()
                    

def uploader_on_change(change): 
    error_output.clear_output()
    with error_output:
        try: 
            upload()
        except Exception as e:
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            print(exc_type, fname, exc_tb.tb_lineno)

uploader.observe(uploader_on_change, names="value")

error_output = widgets.Output(layout=Layout(
    padding="2px", 
    height="100%", 
    align_items='center',
    justify_conten="center",
    width="100%"))
build_experiment = widgets.Button(
    description='Build Experiment',
    disabled=False,
    button_style='info',
    tooltip='Click me',
    layout=Layout(width='auto', height="auto", margin="15px 60px"))

home_view_container = widgets.Box([
    Label("Build a new experiment, or load data from an old one"), 
    widgets.HBox([build_experiment, uploader], 
                layout=widgets.Layout(
                display="flex",
                justify_content="center",
                margin="30px 0",
                align_items="center"
                )),
    error_output,],
    layout=Layout(
    display="flex",
    flex_flow='column',
    width="100%",
    height="100%",
    margin="60px 0",
    justify_content='center',
    align_items="center",))

# display(home_view_container)
# home_view_container.layout.display = 'none'

wrapper_container = widgets.Box([header,main_container,home_view_container],
                               layout=Layout(display="flex",flex_flow="column"))
main_container.layout.display = 'none'
display(wrapper_container)

def build():
    error_output.clear_output()
    with error_output:
        print("501 Not Implemented")
#         start_server()
#         home_view_container.layout.display = 'none'
#         main_container.layout.display = 'inline'
    
def connect(): 
    error_output.clear_output()
    with error_output:
        resume_server()
        
    
def build_experiment_clicked(b): 
    build()

    
def connect_to_database_clicked(b):
    connect()
 
    
build_experiment.on_click(build_experiment_clicked)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 750)